This pairing combines a trend-following indicator with a momentum oscillator to provide confirmation of potential trend changes. 
Moving Average Convergence Divergence (MACD): Identifies momentum and potential trend changes through the crossover of its two moving average lines.
Usage: A bullish crossover (MACD line crosses above the signal line) can signal momentum is shifting to the upside.
Relative Strength Index (RSI): Confirms momentum and helps avoid false signals from the MACD.
Usage: An oversold RSI reading followed by a MACD bullish crossover provides more credible confirmation of a potential trend reversal. 

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279221,85.324368,83.909697,84.556835,118944600
1,2010-01-05,85.504944,85.542570,84.918006,85.226528,111579900
2,2010-01-06,85.565170,85.775865,85.354475,85.422196,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212303,86.249930,85.527544,85.700613,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
df['bullishMACD'] = 0.0
df['bullishMACD'] = np.where(df['macd'] > df['signal'], 1.0, 0.0)
df['crossover_MACD'] = df['bullishMACD'].diff()

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima14'] = df['rsi'].rolling(14).mean()
df['rsi_bullish'] = 0.0
df['rsi_bullish'] = np.where(df['rsi'] > df['rsima14'], 1.0, 0.0)
df['crossover_rsi'] = df['rsi_bullish'].diff()


In [5]:
df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,macd,signal,histogram,bullishMACD,crossover_MACD,rsi,rsima6,rsima14,rsi_bullish,crossover_rsi
0,2010-02-22,83.646332,83.962374,83.398010,83.939801,132346900,-0.677005,-1.432509,0.755503,1.0,1.0,55.167543,51.564427,44.322780,1.0,0.0
1,2010-02-23,82.630455,83.676410,82.412233,83.420567,207497000,-0.617658,-1.269538,0.651880,1.0,0.0,49.034572,52.562844,44.606891,1.0,0.0
2,2010-02-24,83.390457,83.525905,82.668072,82.878767,176350700,-0.503495,-1.116330,0.612835,1.0,0.0,53.224125,53.107436,45.322276,1.0,0.0
3,2010-02-25,83.277573,83.337773,81.975776,82.201518,259634700,-0.417318,-0.976527,0.559209,1.0,0.0,52.533358,53.216282,46.648912,1.0,0.0
4,2010-02-26,83.330284,83.616232,82.856220,83.352858,173589300,-0.340840,-0.849390,0.508550,1.0,0.0,52.841139,52.981806,47.928009,1.0,0.0


In [6]:
# Moving Average Convergence / Divergence (MACD) and RSI Combined Strategy
if __name__ == '__main__':
    
    chart = Chart(title="MACD + RSI Strategy", maximize=True, inner_height=0.7)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart
    chart.set(df)

    # Create MACD subchart
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.15, sync=True)
    macd_line = macd_chart.create_line('MACD', color="#2ee30f", width=1, price_line=False, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('Signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('Histogram', color="#1341e4", price_line=False, price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])

    # Create RSI subchart
    rsi_chart = chart.create_subchart(position='left', width=1.0, height=0.15, sync=True)
    rsi_line = rsi_chart.create_line('RSI', color="#e11d74", width=1, price_line=False, price_label=False)
    rsi_line.set(df[['Date', 'rsi']])
    
    # Add RSI MA lines
    # Create temporary DataFrames with renamed columns for the MA lines
    rsi_ma6_df = df[['Date', 'rsima6']].rename(columns={'rsima6': 'RSI MA(6)'})
    rsi_ma14_df = df[['Date', 'rsima14']].rename(columns={'rsima14': 'RSI MA(14)'})
    
    rsi_ma6 = rsi_chart.create_line('RSI MA(6)', color="#f7e01d", width=1, price_line=False, price_label=False)
    rsi_ma6.set(rsi_ma6_df)
    
    rsi_ma14 = rsi_chart.create_line('RSI MA(14)', color="#1df7eb", width=1, price_line=False, price_label=False)
    rsi_ma14.set(rsi_ma14_df)

    # Add overbought/oversold lines
    overbought_line = rsi_chart.create_line('Overbought', color="#ff0000", width=1, price_line=True, price_label=True)
    overbought_line.set(pd.DataFrame({'Date': df['Date'], 'Overbought': [70] * len(df)}))
    
    oversold_line = rsi_chart.create_line('Oversold', color="#00ff00", width=1, price_line=True, price_label=True)
    oversold_line.set(pd.DataFrame({'Date': df['Date'], 'Oversold': [30] * len(df)}))

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find trading signals
    for i in range(1, len(df)):
        MACD_diff = df.iloc[i]['crossover_MACD']
        RSI_current = df.iloc[i]['rsi']
        current_time = df.iloc[i]['Date']

        # Buy signal: MACD crosses above signal line AND RSI is oversold (< 30)
        if MACD_diff == 1 and RSI_current < 30:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Strong Buy'
            })
        
        # Sell signal: MACD crosses below signal line AND RSI is overbought (> 70)
        elif MACD_diff == -1 and RSI_current > 70:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Strong Sell'
            })

    # Add all markers at once
    if markers:
        chart.marker_list(markers)
    
chart.show(block=True)